In [17]:
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme(style="white")
import os
import os

### Two Car Experiments

Define track lengths

In [18]:
track_lengths = {"track_porto": 57.492612883355136,"racecar_walker": 73.2483359322749,"track_barca": 221.14172044855775}

In [19]:
track_names = {'track_porto':"Porto",'racecar_walker':'Walker','track_barca':"Barca"}
controller_names = {"mpcc":"MPCC","mpc_hype":"MPC Hype","disparity_extender":"DE","pure_pursuit":"Pure Pursuit"}
target_controllers = {"disparity_extender":"DE","pure_pursuit":"Pure Pursuit"}
progress_file_str = "progress_{}_{}_{}_{}_{}.csv"
collision_file_str = "collisions_{}_{}_{}_{}_{}.csv"
start_str = 'two_vehicle_experiments/'
cols={0:"racecar_name",1:"laps_completed",2:"race_time",3:"experiment_number"}
rows = []

interesting_dfs = []
for track in track_names.keys():
    for controller in controller_names.keys():
        for opp in target_controllers.keys():
            for target in target_controllers:
                try:
                    pdf = start_str+progress_file_str.format(track,"false",controller,opp,target)
                    
                    cdf = start_str+collision_file_str.format(track,"false",controller,opp,target)
                   
                        
                    df = pd.read_csv(pdf,header=None).rename(columns=cols)
                    
                    # some of the measurement's are not correct
                    rows_to_drop = df[(df['laps_completed'] > 0.5) & (df['race_time'] < 12)].index.values
                    df.drop(rows_to_drop)
                    
                    # some controller's have perfect safey so there will not be a collision file
                    if(os.path.exists(cdf)):
                        df2 = pd.read_csv(cdf,header=None)[0:30]
                        val = 100 * (1-(df2.count()[0]/30))
                    else:
                        val = 100.0
                    new_df =  df.groupby("racecar_name").median()
                    new_df['efficiency'] = (new_df['laps_completed'] / new_df['race_time']) * track_lengths[track]
                    new_df = new_df.reset_index()
                    row = [track_names[track],controller_names[controller],target_controllers[opp],target_controllers[target],new_df[new_df['racecar_name'] =='racecar2']['efficiency'].values[0],new_df[new_df['racecar_name'] =='racecar']['efficiency'].values[0],new_df['race_time'][0],val]
                    rows.append(row)
                    
                    if(new_df['efficiency'].max()>6):
                        interesting_dfs.append(df)
                except Exception as e:
                    #print(e)
                    continue

In [20]:
df3 = pd.DataFrame(rows, columns =['Track', 'Approach','Opponent Controller','Local Planner','Ego Efficiency','Opponent Efficiency','Race Duration (s)','Safety'])
df3 = round(df3,2)
df3 = df3.set_index('Track').sort_values(by=["Track","Approach"])

In [21]:
print(df3.to_latex())

\begin{tabular}{llllrrrr}
\toprule
{} &      Approach & Opponent Controller & Local Planner &  Ego Efficiency &  Opponent Efficiency &  Race Duration (s) &  Safety \\
Track  &               &                     &               &                 &                      &                    &         \\
\midrule
Barca  &            DE &                  DE &            DE &            5.12 &                 4.50 &               7.63 &    0.00 \\
Barca  &            DE &                  DE &  Pure Pursuit &            5.16 &                 4.47 &               7.69 &    0.00 \\
Barca  &      MPC Hype &                  DE &            DE &            0.00 &                 5.85 &               5.47 &    0.00 \\
Barca  &      MPC Hype &                  DE &  Pure Pursuit &            0.06 &                 5.74 &               5.49 &    0.00 \\
Barca  &          MPCC &                  DE &            DE &            3.18 &                 3.01 &              11.26 &    0.00 \\
Barca  &

### Three Car Experiments

In [22]:
track_names = {'track_porto':"Porto",'racecar_walker':'Walker','track_barca':"Barca"}
controller_names = {"mpcc":"MPCC","mpc_hype":"MPC Hype","disparity_extender":"DE","pure_pursuit":"Pure Pursuit"}
target_controllers = {"disparity_extender":"DE","pure_pursuit":"Pure Pursuit"}
progress_file_str = "progress_{}_{}_{}_{}_{}.csv"
collision_file_str = "collisions_{}_{}_{}_{}_{}.csv"
start_str = 'three_vehicle_experiments/'
cols={0:"racecar_name",1:"laps_completed",2:"race_time",3:"experiment_number"}
rows = []

interesting_dfs = []
for track in track_names.keys():
    for controller in controller_names.keys():
        for opp in target_controllers.keys():
            for target in target_controllers:
                try:
                    pdf = start_str+progress_file_str.format(track,"false",controller,opp,target)
                    
                    cdf = start_str+collision_file_str.format(track,"false",controller,opp,target)
                   
                        
                    df = pd.read_csv(pdf,header=None).rename(columns=cols)
                    
                    # some of the measurement's are not correct
                    rows_to_drop = df[(df['laps_completed'] > 0.5) & (df['race_time'] < 12)].index.values
                    df.drop(rows_to_drop)
                    
                    # some controller's have perfect safey so there will not be a collision file
                    if(os.path.exists(cdf)):
                        df2 = pd.read_csv(cdf,header=None)[0:30]
                        val = 100 * (1-(df2.count()[0]/30))
                    else:
                        val = 100.0
                    new_df =  df.groupby("racecar_name").median()
                    new_df['efficiency'] = (new_df['laps_completed'] / new_df['race_time']) * track_lengths[track]
                    new_df = new_df.reset_index()
                    row = [track_names[track],controller_names[controller],target_controllers[opp],target_controllers[target],new_df[new_df['racecar_name'] =='racecar2']['efficiency'].values[0],new_df[new_df['racecar_name'] =='racecar']['efficiency'].values[0],new_df['race_time'][0],val]
                    rows.append(row)
                    
                    if(new_df['efficiency'].max()>6):
                        interesting_dfs.append(df)
                except Exception as e:
                    #print(e)
                    continue

In [23]:
df3 = pd.DataFrame(rows, columns =['Track', 'Approach','Opponent Controller','Local Planner','Ego Efficiency','Opponent Efficiency','Race Duration (s)','Safety'])
df3 = round(df3,2)
df3 = df3.set_index('Track').sort_values(by=["Track","Approach"])

In [25]:
print(df3.to_latex())

\begin{tabular}{llllrrrr}
\toprule
{} &      Approach & Opponent Controller & Local Planner &  Ego Efficiency &  Opponent Efficiency &  Race Duration (s) &  Safety \\
Track  &               &                     &               &                 &                      &                    &         \\
\midrule
Barca  &            DE &                  DE &            DE &            5.06 &                 4.60 &               6.09 &    0.00 \\
Barca  &            DE &                  DE &  Pure Pursuit &            5.04 &                 4.57 &               6.05 &    0.00 \\
Barca  &      MPC Hype &                  DE &            DE &            0.69 &                 5.11 &               5.37 &    0.00 \\
Barca  &      MPC Hype &                  DE &  Pure Pursuit &            0.09 &                 4.64 &               5.36 &    0.00 \\
Barca  &          MPCC &                  DE &            DE &            3.30 &                 2.70 &              10.60 &    0.00 \\
Barca  &